<a href="https://colab.research.google.com/github/ArghyaPal/ONWARD/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## All libraries

In [0]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
from torchvision.utils import save_image
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
#from IPython.display import HTML
from model import *
#from dataloader import *
from hyper import *

import os
import h5py
import numpy as np

## Data Loading

In [0]:
# Set random seem for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)


def load_h5data(file_name, desc, dtype):
    h5 = h5py.File(file_name, 'r')
    return np.array(h5.get(desc), dtype=dtype)

def reshape_image(image_array):
    return image_array.transpose((0,3,1,2))

def reshape_labels(label_array):
    n, h, w, c = label_array.shape
    return label_array.reshape((n))

#specify your data path in 'path'
path = '/groups2/gaa50088/pathology/pcamv1/'
train_images = load_h5data(os.path.join(path, 'camelyonpatch_level_2_split_train_x.h5'), 'x', np.uint8)
train_images = reshape_image(train_images)
print(train_images.shape)

train_labels = load_h5data(os.path.join(path,'camelyonpatch_level_2_split_train_y.h5'), 'y', np.uint8)
train_labels = reshape_labels(train_labels)

train_images = np.asarray(train_images, np.float32)
train_images /= 255

train_data = torch.tensor(train_images, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)

real_batch = train_data[:32]
print("Eito")
save_image(real_batch, "images/real.png", normalize=True)

train_set = torch.utils.data.TensorDataset(train_data)#, train_labels) #for classification
dataloader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True, drop_last=True)

In [0]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

## Weight initialization

In [0]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

## Making Object of Encoder, Discriminator and Decoder Classes

In [0]:
# Create the encoder
netE = Encoder(ngpu).to(device)
# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netE = nn.DataParallel(netE, list(range(ngpu)))
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netE.apply(weights_init)
# Print the model
#print(netG)



# Create the generator
netG = Generator(ngpu).to(device)
# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)
# Print the model
#print(netG)


# Create the Discriminator
netD = Discriminator(ngpu).to(device)
# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)
# Print the model
#print(netD)

## Loss Definition

In [0]:
# Initialize BCELoss function
criterion = nn.BCELoss()
adversarial_loss = nn.MSELoss()


# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(100, nz, 1, 1, device=device)


# Establish convention for real and fake labels during training
real_label = 1
fake_label = 0


# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerE = optim.Adam(netE.parameters(), lr=lr, betas=(beta1, 0.999))

# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 100

## Start Training Loop

In [0]:
print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader):#, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device) #(64,3,96,96)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, device=device)  # 64 elements
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        
        
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        #print(noise.shape)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        #err_MSE_real = adversarial_loss(output, label)
        #errD_fake = errD_dis + err_MSE_real


        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update E network: maximize log(D(G(z)))
        ###########################
        netE.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        encoded = netE(real_cpu).view(-1)
        #print(encoded.shape)
        encoded = encoded.view(100,100,1,1)
        #print(encoded.shape)
        #encoded = torch.tensor(b_size, encoded, 1, 1, device=device)
        decoded = netG(encoded)
        output = netD(decoded.detach()).view(-1)
        # Calculate G's loss based on this output
        errE = criterion(output, label)
        errG = criterion(output, label)
        # Calculate gradients for G
        errE.backward(retain_graph=True)
        errG.backward(retain_graph=True)
        D_G_z3 = output.mean().item()
        # Update E
        optimizerE.step()
        optimizerG.step()
        
        
        ###########################
        # (3) Update G network: maximize loss of encoder
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward(retain_graph=True)
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()


        # Output training stats
        if i % 10 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        if epoch % 50 == 0:
            fake = netG(fixed_noise).detach().cpu()
            fake_batch = fake[:32]
            save_image(fake_batch, "images/fake.png", normalize=True)
            #for j in range(1,90):
            #    save_image(fake.data[j,:,:,:], "images/%d.png" % j, nrow=1, normalize=True)

        iters += 1